# Ortogonalização, Double Machine Learning e CATE

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Ortogonalização
  * Aplicação do Procedimento de Ortogonalização no Python
* DML - Orthogonal/Double Machine Learning
  * Aplicação do DML no Python
* Arcabouço de Resultados Potenciais, Individual Treatment Effects (ITE) e Conditional Average Treatment Effects (CATE)
  * Estimação do CATE com DML no Python


## Referências

**Principais:**
* Chernozhukov, V., Chetverikov, D., Demirer, M., Duflo, E., Hansen, C., Newey, W., & Robins, J. (2018). Double/debiased machine learning for treatment and structural parameters. The Econometrics Journal, Volume 21, Issue 1, 1 February 2018, Pages C1–C68, https://doi.org/10.1111/ectj.12097
  
* Microsoft EconML: https://econml.azurewebsites.net/
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* DoubleML for python: https://github.com/DoubleML/doubleml-for-py ou https://docs.doubleml.org/stable/index.html

**Complementares:**
* Chernozhukov, V. and C. Hansen (2004). The effects of 401 (k) participation on the wealth distribution: an instrumental quantile regression analysis. Review of Economics and Statistics 86, 735–51. 
* Chernozhukov, V., D. Chetverikov and K. Kato (2014). Gaussian approximation of suprema of empirical processes. Annals of Statistics 42, 1564–97. 
* Chernozhukov, V., J. Escanciano, H. Ichimura, W. Newey and J. Robins (2016). Locally robust semiparametric estimation. Preprint (arXiv:1608.00033). 
* Chernozhukov, V., C. Hansen and M. Spindler (2015a). Post-selection and post-regularization inference in linear models with very many controls and instruments. Americal Economic Review: Papers and Proceedings 105, 486–90. 
* Chernozhukov, V., C. Hansen and M. Spindler (2015b). Valid post-selection and post-regularization inference: an elementary, general approach. Annual Review of Economics 7, 649–88.

## Ortogonalização

A ideia de ortogonalização é baseada em um teorema elaborado por três econometristas em 1933, Ragnar Frisch, Frederick V. Waugh e Michael C. Lovell. Simplificando, afirma que você pode decompor qualquer modelo de regressão linear multivariável em três estágios ou modelos. 

Digamos que você tem uma matriz de covariáveis $X$, e voce particiona ela em duas partes, $X_{1}$ e $D$. 

* **Primeira Etapa**

Pegamos o primeiro conjunto de variáveis $X_{1}$ e fazemos uma regressão linear de $X_{1}$ em $Y$, onde $\theta_{1}$ é o vetor de parâmetros

$$ y_{i} = \theta_{0} + \theta_{1} X_{1i} + \epsilon_{i}$$

e guardamos os resíduos dessa regressão ($y^{*}$).

$$ y^{*}_{i} = y_{i} - \hat{y}_{i} = y_{i} - ( \hat{\theta}_{0} + \hat{\theta}_{1} X_{1i} )$$

* **Segunda Etapa**

Pegamos novamente o primeiro conjunto de características, mas agora executamos um modelo onde estimamos o segundo conjunto de características ($D$)

$$ D_{i} = \gamma_{0} + \gamma_{1} X_{1i} + e_{i}$$

Aqui, estamos usando o primeiro conjunto de recursos para prever o segundo conjunto de recursos. Por fim, consideramos também os resíduos desta segunda etapa.

$$ D_{i}^{*} = D_{i} - (\hat{\gamma_{0}} + \hat{\gamma_{1}} X_{1i})$$

* **Terceira etapa**

Por fim, pegamos os resíduos do primeiro e do segundo estágio e estimamos o seguinte modelo

$$ y_{i}^{*} = \alpha_{0} + \beta_{2} D_{i}^{*} + e_{i}$$


* **Teorema Frisch – Waugh – Lovell (FWL)**

O teorema FWL afirma que a estimativa do parâmetro $\hat{\beta}_{2}$, estimado anteriormente, é equivalente ao que obtemos ao executar a regressão completa, com todas as covariáveis.

$$ y_{i} = \beta_{0} + \beta_{1} X_{1i} + \beta_{2} D_{i} + e_{i}$$


**Intuição do teorema FWL**

Sabemos que a regressão é um modelo muito especial. Cada um de seus parâmetros tem a interpretação de uma derivada parcial, quanto seria Y se X aumentasse em uma unidade, mantendo todos as outras covariáveis constantes. Sabemos também que se omitirmos variáveis ​​da regressão, teremos viés. Especificamente, viés variável omitido (ou viés de confusão). Ainda assim, Frisch-Waugh-Lovell está dizendo que posso dividir meu modelo de regressão em duas partes, nenhuma delas contendo o conjunto completo de recursos, e ainda assim obter a mesma estimativa que obteria executando a regressão inteira. 

O teorema fornece algumas dicas sobre o que a regressão linear está fazendo. Para obter o coeficiente de uma variável $X_{k}$, a regressão primeiro usa todas as outras variáveis ​​para prever $X_{k}$ e pega os resíduos. Isso “limpa” $X_{k}$ de qualquer influência dessas variáveis. Dessa forma, quando tentamos entender o impacto de $X_{k}$ sobre $Y$, estará livre de viés de variável omitida. Em segundo lugar, a regressão usa todas as outras variáveis ​​para prever $Y$ e pega os resíduos. Isso “limpa” $Y$ de qualquer influência dessas variáveis, reduzindo a variância de $Y$ para que seja mais fácil ver como $X_{k}$ afeta $Y$.

A regressão linear está estimando o impacto de $D$ sobre $y$ enquanto contabiliza $X_{1}$. Isso é importante para inferência causal. 

Assim, podemos construir um modelo que preveja um tratamento ($D$) usando as covariáveis $X$, um modelo que prevê o resultado $y$ usando as mesmas covariáveis, pegar os resíduos de ambos os modelos e executar um modelo que estime como o resíduo de $D$ afetam os resíduos de $y$. Este último modelo vai me dizer como $D$ afeta $y$ enquanto controla por $X$. Ou seja, **os dois primeiros modelos controlam as variáveis de confusão**. Eles estão **gerando dados que são praticamente aleatórios**. Lembre que é isso que estaria distorcendo seus dados. Então, usamos no modelo final para estimar o efeito causal de interesse.

### Aplicação do Procedimento de Ortogonalização no Python

Vamos aplicar o procedimento de ortogonalização considerando um modelo de regressão linear simples. Vamos realizar a orgonalização supondo linearidade entre as variáveis para entender o conceito. Posteriormente, vamos aplicar o procedimento de ortogonalização em um modelo de machine learning. 

In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [19]:
# DataFrame
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

# Criar a variável de resultado
df['Y'] = df['bweight']

# Criar a variável 'Treated' com valor 1 se 'mbsmoke' for 'smoker', caso contrário 0
df['D'] = np.where(df['mbsmoke'] == 'smoker', 1, 0)

# Criar a variável 'casada' com valor 1 se 'mmarried' for 'married', caso contrário 0
df['casada'] = np.where(df['mmarried'] == 'married', 1, 0)


Para desviar este conjunto de dados, precisaremos de dois modelos. O primeiro modelo, vamos chamá-lo $M_{D}(X)$, prevê o tratamento (Se a gestante é fumante, no nosso caso) utilizando os confundidores. É um dos estágios que vimos acima, no teorema de Frisch–Waugh–Lovell.

Assim que tivermos este modelo, construiremos os resíduos

$$ D_{i}^{*} = D_{i} - M_{D}(X_{i})$$

In [20]:
m_D = smf.ols("D ~ 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['D_star'] = df['D'] - m_D.predict(df)

Você pode pensar neste resíduo como uma versão do tratamento que é imparcial ou, melhor ainda, que é impossível de prever a partir dos fatores de confusão $X$. Como os fatores de confusão já eram usados ​​para prever $D$, o resíduo é, por definição, imprevisível com com $X$. Outra maneira de dizer isso é que o viés foi explicado pelo modelo $M_{D}(X)$, produzindo $D_{i}^{*}$ que é tão bom quanto atribuído aleatoriamente. É claro que isso só funciona se tivermos em $X$ todos os fatores de confusão que causam ambos $D$ e $Y$.

Também podemos construir resíduos para o resultado.

$$ y_{i}^{*} = y_{i} - M_{y}(X_{i})$$


Este é outro estágio do teorema de Frisch – Waugh – Lovell. Isso não torna o conjunto menos tendencioso, mas facilita a estimativa do efeito, reduzindo a variância em $y$. Mais uma vez você pode pensar $y_{i}^{*}$ como uma versão de $y_{i}$ imprevisível de $X$ ou que teve todas as suas variações devido a $X$ explicadas. Pense nisso. Nós já usamos $X$ para prever $y$ com $M_{y}(X_{i})$. E $y_{i}^{*}$ é o erro dessa previsão. Então, por definição, não é possível prever isso a partir de $X$. Todas as informações em $X$ para prever $y$ já foram usadas. Se for esse o caso, a única coisa que resta para explicar $y_{i}^{*}$ é algo que não usamos usamos para construí-lo (não incluído em $X$), que é apenas o tratamento (novamente, assumindo que não há fatores de confusão não medidos).


In [21]:
m_y = smf.ols("Y ~  1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['y_star'] = df['Y'] - m_y.predict(df)

Por fim, aplicando o teorema:

In [22]:
FWL1 = smf.ols("y_star ~ D_star", data=df).fit()
print(FWL1.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_star   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     100.6
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           1.91e-23
Time:                        16:44:21   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.172e+04
Df Residuals:                    4640   BIC:                         7.173e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.586e-11      8.041   1.97e-12      1.0

Podemos comparar o resultado com o OLS tradicional com covariáveis:

In [23]:
ols = smf.ols("Y ~ D + 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     38.49
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          5.80e-100
Time:                        16:44:25   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.175e+04
Df Residuals:                    4627   BIC:                         7.184e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2851.5203     54.983     51.861      0.0

Realmente o teorema de Frisch-Waugh-Lovell funciona mesmo.

Podemos concluir que depois de fazermos as duas transformações, a única coisa que resta para prever esses resíduos é o tratamento. 

Para resumir, ao prever o tratamento, construímos $D^{*}$ que funciona como uma versão imparcial do tratamento; ao prever o resultado, construímos $y^{*}$ que é uma versão do resultado que só pode ser explicada se usarmos o tratamento. Esses dados, onde substituímos por $y$ por $y^{*}$ e $D$ por $D^{*}$, são os dados desviados que queríamos. Podemos usá-lo para avaliar nosso modelo causal da mesma forma que fizemos anteriormente, usando dados aleatórios.

##  DML - Orthogonal/Double Machine Learning

* Double Machine Learning - DML
* Double/Debiased Machine Learning - DDML 

Quando temos muitas possíveis variáveis ​​de controle, podemos querer selecionar as mais relevantes, possivelmente capturando não linearidades e interações. Algoritmos de aprendizado de máquina são perfeitos para essa tarefa. No entanto, nesses casos, estamos introduzindo um viés que é chamado de regularização ou pré-teste, ou viés de seleção de recursos ($X$). 

Ou seja, o que acontece se a dimensão de $X$ aumenta e não conhecemos a forma funcional através da qual $X$ afeta $Y$ e $D$?

Nesses casos, podemos usar **algoritmos de aprendizado de máquina** para **descobrir essas relações não lineares de alta dimensão**.

No artigo de **Chernozhukov et al (2018)**, os autores mostraram que também é possível fazer ortogonalização com modelos de aprendizado de máquina. 

$$ y_{i}^{*} = y_{i} - M_{y}(X_{i})$$

$$ D_{i}^{*} = D_{i} - M_{D}(X_{i})$$

onde $M_{y}$ e $M_{D}$ são modelos de aprendizado de máquina.


***Machine Learning (ML)* e *Overfitting*** : Os modelos de aprendizado de máquina (ML) podem ajustar-se perfeitamente aos dados, ou melhor, superajustá-los (*Overfitting* / "sobreajuste"). 

Apenas olhando para as equações anteriores, podemos saber o que acontecerá nesse caso.
  * Se $M_{y}$ de alguma forma, os resíduos serão todos muito próximos de zero. Se isso acontecer, será difícil descobrir como $D$ afeta isso. 
  * Se $M_{D}$ de alguma forma superajusta, seus resíduos também serão próximos de zero. Conseqüentemente, não haverá variação no resíduo do tratamento para ver como isso pode impactar o resultado.

O que fazer em caso de *Overfitting*?

* **Regularização**: técnica que adiciona um termo à função de perda que penaliza os coeficientes do modelo. Isso faz com que o modelo seja menos sensível aos dados de treinamento, evitando o superajuste.

Quais os modelos de ML mais comuns para previsão?

* Random Forest: é um modelo de aprendizado de máquina que pode ser usado tanto para classificação quanto para regressão. Ele é um modelo de conjunto que treina várias árvores de decisão em subconjuntos aleatórios dos dados e faz a média de suas previsões.
* Gradient Boosting: é um modelo de aprendizado de máquina que constrói um modelo aditivo de forma progressiva. Ele permite a otimização de funções de perda diferenciáveis arbitrárias.
* Redes Neurais: são modelos de aprendizado de máquina que são inspirados na forma como o cérebro humano funciona. Eles são compostos por camadas de neurônios que processam e transmitem informações.
* Support Vector Machines: são modelos de aprendizado de máquina que são usados para classificação e regressão. Eles são eficazes em espaços de alta dimensão e são capazes de lidar com dados não lineares.
* Outros modelos de ML mais usados em economia são: LASSO, Ridge, Elastic Net, etc. LASSO é um método de regressão que adiciona uma penalidade L1 à função de perda. Isso faz com que alguns coeficientes sejam exatamente zero, o que é útil para seleção de recursos. Ridge é um método de regressão que adiciona uma penalidade L2 à função de perda. Isso faz com que os coeficientes sejam menores, o que é útil para reduzir a variância. Elastic Net é um método de regressão que combina as penalidades L1 e L2. Isso permite que você selecione recursos e reduza a variância ao mesmo tempo.

**Validação Cruzada (Cross-Validation)**:  é uma técnica estatística usada para avaliar a performance de um modelo preditivo e sua capacidade de generalização para novos dados. Em essência, é um método de dividir os dados disponíveis em partes para treinar e testar o modelo de maneira eficiente e confiável.

* **Validação Cruzada K-fold**: é uma técnica de validação cruzada que divide os dados em k partes. O modelo é treinado em k-1 partes e testado na parte restante. Isso é feito k vezes, de modo que cada parte seja usada como conjunto de teste exatamente uma vez.

<div style="text-align:center;">
    <img src="images\kfold-cv.png"  alt="Imagem" style="width: 450px;"/>
</div>

Felizmente, esse tipo de validação cruzada é muito fácil de implementar usando `cross_val_predicta` função do Sklearn.


**Aplicações do DML**

Visto esses conceitos básicos, vamos voltar ao DML. Especialmente, vamos aplicar o DML considerando dois modelos de machine learning. Primeiramente consideramos o Random Forest e posteriormente o Gradient Boosting.

1. Aplicação do DML com o Random Forest

In [83]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "D"
y = "Y"

folds = 8
np.random.seed(123)

m_D1 = RandomForestClassifier(n_estimators=100, random_state=123)
D_res1 = df[D] - cross_val_predict(m_D1, df[X], df[D], cv=folds)

m_y1 = RandomForestRegressor(n_estimators=100, random_state=123)
y_res1 = df[y] - cross_val_predict(m_y1, df[X], df[y], cv=folds)

In [84]:
FWL_DML1 = smf.ols("y_res1 ~ D_res1", data=df).fit()
print(FWL_DML1.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res1   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     56.04
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           8.43e-14
Time:                        18:10:55   Log-Likelihood:                -36202.
No. Observations:                4642   AIC:                         7.241e+04
Df Residuals:                    4640   BIC:                         7.242e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.1647      8.898      2.716      0.0

2. Aplicação do DML com o Gradient Boosting Machines (GBM)

In [108]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import numpy as np

# Definir as variáveis X, D, y
X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "D"
y = "Y"

# Definir o número de folds para a validação cruzada
folds = 8
np.random.seed(123)

# Garantir reprodutibilidade nos modelos de Gradient Boosting
m_D2 = GradientBoostingClassifier(n_estimators=100, random_state=123)
D_res2 = df[D] - cross_val_predict(m_D2, df[X], df[D], cv=folds)

m_y2 = GradientBoostingRegressor(n_estimators=100, random_state=123)
y_res2 = df[y] - cross_val_predict(m_y2, df[X], df[y], cv=folds)

O segundo estágio do DML é estimar o efeito do tratamento no resultado. Para isso, usamos um modelo de regressão linear.

In [109]:
FWL_DML2 = smf.ols("y_res2 ~ D_res2", data=df).fit()
print(FWL_DML2.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res2   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     66.85
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           3.74e-16
Time:                        18:22:03   Log-Likelihood:                -35892.
No. Observations:                4642   AIC:                         7.179e+04
Df Residuals:                    4640   BIC:                         7.180e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.2145      8.556      2.713      0.0

Vimos a mecânica do DML. Com o uso de modelos de ML não lineares.

Na aula extra (13-xtr_ML_overf_CV.ipynb), você pode verificar como escolher o melhor modelo de ML (procedimentos de validação cruzada, e critérios de seleção de modelos).



**Aplicações do DML com pacote EconML**

Vamos utilizar o pacote EconML para aplicar o DML de forma mais eficiente. Esse pacote aplica Validação Cruzada Adaptativa.

In [110]:
import pandas as pd
from econml.dml import LinearDML
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

# Definir as variáveis
X = df[['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']]
D = df['D']
y = df['Y']

# Converter variáveis categóricas em dummies (se necessário)
X = pd.get_dummies(X, drop_first=True)

# Definir os modelos de Machine Learning para (i) X em Y, e (ii) X em D
model_y = GradientBoostingRegressor(n_estimators=100, random_state=123)
model_d = GradientBoostingClassifier(n_estimators=100, random_state=123)

# Criar o estimador LinearDML
estimator = LinearDML(model_y=model_y,
                      model_t=model_d,
                      discrete_treatment=True,
                      cv=5,
                      random_state=123)

# Ajustar o modelo
estimator.fit(y, D, X=X)

ate_inf = estimator.ate_inference(X=X)
print(ate_inf.summary())

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
  -206.172      25.877 -7.967    0.0      -256.891      -155.454
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
   124.36        -395.601         160.819
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
     127.024       -445.011        185.398
------------------------------------------


Os resultados não são identicos porque No método manual com cross_val_predict, você está usando validação cruzada *k-fold* para calcular os resíduos. O cross_val_predict faz predições out-of-sample para cada observação, mas: 
* Ele não ajusta automaticamente os hiperparâmetros dos modelos.
* Ele aplica a validação cruzada de forma isolada para Y e D, sem sincronizar as divisões.

O LinearML, por outro lado:
* Usa validação cruzada para ambos os modelos $M_{y}$ e $M_{D}$,, mas com divisões sincronizadas, garantindo que os conjuntos de treino e teste sejam consistentes entre Y e D.
* Possui ajustes internos que minimizam o viés e a variância nos resíduos.

Essas diferenças podem levar a discrepâncias nos resíduos $y^{*}$ e $D^{*}$, e, consequentemente, nos coeficientes estimados.

Quanto aos erros padrão do segundo estágio, 
* No método manual, o segundo estágio (regressão linear com smf.ols) assume que os resíduos $Y^{*}$ e $D^{*}$ são independentes e identicamente distribuídos (i.i.d.). Isso pode subestimar ou superestimar a variância de $\theta$.
* O LinearDML ajusta os erros padrão para levar em conta:
  * A incerteza nos modelos do primeiro estágio ($M_{y}$ e $M_{D}$).
  * A possível correlação entre $Y^{*}$ e $D^{*}$ devido a predições do primeiro estágio.

Esse ajuste no LinearDML pode resultar em coeficientes ligeiramente diferentes e intervalos de confiança mais amplos.

## Arcabouço de Resultados Potenciais, Individual Treatment Effects (ITE) e Conditional Average Treatment Effects (CATE)


**Individual Treatment Effect - ITE - $\beta_{i}$**

Podemos definir o efeito do tratamento individual (ITE) como a diferença entre os resultados potenciais. 

$$ 
\beta_{i}^{ITE} = Y_{i}(1) - Y_{i}(0) 
$$

Segundo o problema fundamental da inferência causal, nunca podemos observar o mesmo indivíduo sob diferentes condições de tratamento. 

$$
Y^{obs}_i(D)= 
\begin{cases}
Y_i(1), & \text{se } D=1 \\
Y_i(0), & \text{se } D=0
\end{cases}
$$


**Average Treatment Effect - ATE**

Como estávamos acostumados, pode-se definir o efeito médio do tratamento como:

$$
\beta^{ATE}= E[Y_i(1) − Y_i(0)] = E [ \beta_{i}^{ITE} ] = E[\beta_i]
$$

**Conditional Average Treatment Effect - CATE**

e o efeito do tratamento médio condicional (CATE) como:


$$ \beta^{CATE}(x) = E[Y_i(1) − Y_i(0)|X] = E[\beta_i|X_{i}=x] $$

O CATE é a média do efeito do tratamento para indivíduos com características específicas representadas pelas covariáveis $X$.

Repare que **os ITE são inerentemente não observáveis.** Entretanto, o que pode ser estimado, em vez disso, é o **Conditional Average Treatment Effect (CATE)**. Ou seja, o efeito esperado do tratamento individual, condicional em covariáveis $​​X$. 

Vejamos os pressupostos necessários para identificação do efeito causal.


**Hipóteses de Identificação do CATE**

São as condições necessárias para garantir que possamos identificar e estimar o CATE usando dados observacionais. Essas hipóteses estabelecem como o efeito causal pode ser extraído, evitando viés e problemas de endogeneidade. 

* **Não Confundimento** / Inconfundibilidade (Unconfoundedness): Justifica que todas as diferenças entre tratados e não tratados são capturadas pelas covariáveis

$$ Y_{i}(0), Y_{i}(1) \perp T|X $$

* **Sobreposição** (Overlap): Garante que temos dados para comparar indivíduos semelhantes em ambos os grupos.
  * Para cada valor das covariáveis $X$, deve haver unidades suficientes tanto no grupo tratado quanto no grupo controle. Sem essa suposição, não seria possível comparar grupos semelhantes sob diferentes níveis de tratamento.
  
$$ 0 < P(D_{i}=1| X_{i}=x) < 1 $$


* **Consistência** (Consistency): A suposição de consistência estabelece que o resultado observado ($Y_{i}^{obs}$) para uma unidade corresponde ao resultado potencial associado ao tratamento efetivamente recebido:

$$
Y^{obs}_i(D)= 
\begin{cases}
Y_i(1), & \text{se } D=1 \\
Y_i(0), & \text{se } D=0
\end{cases}
$$

Essa hipótese conecta os resultados observados aos contrafactuais de forma coerente e supõe que:
* Não há múltiplas versões do tratamento ($D$) que possam gerar diferentes resultados potenciais.
* O tratamento recebido por uma unidade não afeta os resultados de outras unidades (parte do SUTVA).



**E o como conectar o DML com o CATE?**

Uma primeira argumentação é que os modelos tradicionais lineares são limitados, principalmente a suposição de linearidade de $Y$ em $X$ (OLS não forneceria um modelo consistente). Os pesquisadores argumentam que a linearidade é uma suposição forte e que os modelos de machine learning são mais flexíveis. Ou seja, seria ótimo se os pesquisadores pudessem substituir o modelo linear por um modelo de machine learning mais flexível. 

Como argumenta Chernozhukov et al (2018), pelo teorema de Frisch-Waugh-Lovell, pode-se substituir o modelo linear por um modelo de machine learning. Logo, podemos conectar o tratamento (D) como os recursos (X) a um modelo de ML específico (Decision Trees, rede neural ou Gradient Boosting). 

$$ y_{i} = M(X_{i}, T_{i}) + \epsilon_{i} $$


mas a partir daí, não está claro como podemos obter estimativas do efeito do tratamento, uma vez que este modelo produzirá previsões de $Y$, e não de $\beta$. Vamos aprender uns conceitos importantes para solucionar essa questão.




### Estimação do CATE com DML

Até agora, vimos como o Double/Debiased ML (Double Machine Learning - DML) nos permite focar na estimativa do Efeito Médio do Tratamento (ATE). 

No entanto, ele também pode ser usado para estimar a **heterogeneidade dos efeitos do tratamento** ou o **Efeito Médio Condicional do Tratamento (CATE)**. Ou seja, em vez de considerar um efeito de tratamento constante para todas as observações, o modelo ajustado permite um efeito diferente com base em suas características específicas. Isto é, enquanto o ATE assume que o tratamento tem o mesmo efeito em todos os indivíduos, o CATE permite que o efeito varie de acordo com as características individuais, capturadas por $X$.

Essa ideia é trabalhada no artigo de Chernozhukov et al (2018), que propõe uma abordagem para estimar o CATE usando DML.


<div style="text-align:center;">
    <img src="images\Chernozhucov.png"  alt="Imagem" style="width: 500px;"/>
</div>




Com base no CATE, podemos definir a seguinte equação:

$$ Y = \theta(X)D + g_{y}(X) + \epsilon$$

* $\theta(X)$ é o efeito heterogêneo do tratamento (CATE).
* $g_{y}(X)$ é uma função de $X$ que captura a relação entre $Y$ e $X$ (ou seja, os fatores não relacionados ao tratamento).
* $E(\epsilon|D,X)=0$, e

$$ D = m_{D}(X) + \eta$$

* $m_{D}(X)$ é uma função de $X$ que captura a relação entre $D$ e $X$
* $E(\eta|X)=0$.

Isso significa que o efeito do tratamento ($\theta(X)$) é uma função de $X$, e não um valor constante. Além disso,  tanto o tratamento ($D$) quanto o resultado ($Y$) são funções de $X$.


Utilizando o DML, podemos estimar o CATE seguindo os seguintes passos:

Com base no procedimento de ortogonalização, os primeiros estágios do DML são:

$$ D^{*} = D - m_{0}(X)$$

e,

$$ Y^{*} = Y - g_{y}(X)$$

Posteriormente, teremos o segundo estágio do DML:

$$ Y^{*} = \theta(X)D^{*} + \epsilon$$


No primeiro estágio do DML, ajustamos modelos para $D$ e $Y$ em função de $X$, gerando os resíduos $D^{∗}$ e $Y^{∗}$. Esses resíduos removem a dependência de $X$, permitindo que a segunda etapa estime o efeito causal sem viés. Então, regredindo $Y^{*}$ em $D^{*}$, obtemos o CATE.

$$ \theta = argmin_{\theta} E[(Y^{*} - \theta (X) D^{*})^{2}] + \lambda R(\theta)$$

para algum termo de regularização fortemente convexo $R$, e $\lambda > 0$ é um parâmetro de regularização. O termo de regularização ajuda a evitar overfitting e é especialmente útil em modelos de alta dimensão, onde $\theta$ pode ser esparso (muitos coeficientes iguais a zero, em outras palavras, apenas algumas variáveis ou características têm influência significativa, enquanto o restante pode ser ignorado sem perda relevante de informação).

* Chernozhukov et al (2016) considera o caso em que $\theta(X)$ é uma constante (ATE), ou uma função linear de $X$ (CATE) de baixa dimensão.
* Nie (2017) cai em um Espaço de Hilbert do Kernel Reprodutor (RKHS).
* Chernozhukov et al (2018) consideram o caso de um espaço linear esparso de alta dimensão, onde $\theta(X) = <\theta, \phi(X)>$ para algum mapeamento de características de alta dimensão conhecido e onde $\theta$ tem muito poucas entradas diferentes de zero (esparsas)
* Athey (2019), entre outros resultados, considera o caso em que $\theta(X)$ é uma função lipschitz não paramétrica e usa modelos de floresta aleatória para ajustar a função. Esse métodos permite maior flexibilidade para modelar $\theta(X)$ sem assumir linearidade.
* Foster (2019) permite modelos arbitrários $\theta(X)$ e fornecer resultados com base em medidas de complexidade de amostra do espaço do modelo (por exemplo, complexidade de Rademacher, entropia métrica). Esse métodos permite maior flexibilidade para modelar $\theta(X)$ sem assumir linearidade.

A principal vantagem do DML é que se fizermos suposições paramétricas sobre $\theta(X)$, então se obtém taxas de estimativa rápidas e, para muitos casos de estimadores de estágio final, também normalidade assintótica na estimativa do segundo estágio $\hat{\theta}$.

O ponto novo na nossa abordagem é que, agora, o $\theta(X)$ é o efeito heterogêneo do tratamento (CATE), e assumimos que ele segue uma relação linear das covariáveis/recursos (a linearidade assumida para $\theta(X)$ é uma simplificação útil para interpretação e análise inicial), da seguinte forma:

$$ \theta(X) = X'\beta + \theta_{\text{intercept}}$$

* $X'$ é o vetor transposto das covariáveis.
* $\beta$ são os coeficientes que medem o efeito de cada covariável em $\theta(X)$.
* $\theta_{\text{intercept}}$ é o intercepto do modelo (cate_intercept).

No exemplo anterior eu havia calculado o efeito médio. Agora, vamos calcular o efeito heterogêneo do tratamento (CATE) para cada característica. Os resultados já estão disponíveis no objeto estimator.

In [111]:
print(estimator.summary())

                      Coefficient Results                       
          point_estimate  stderr zstat  pvalue ci_lower ci_upper
----------------------------------------------------------------
casada             20.98  55.278   0.38  0.704  -87.363  129.323
mage             -10.375   5.158 -2.012  0.044  -20.484   -0.266
medu              22.234  10.622  2.093  0.036    1.416   43.052
fhisp            139.822 130.726   1.07  0.285 -116.395   396.04
mhisp           -164.075  150.64 -1.089  0.276 -459.323  131.174
foreign          379.897 143.079  2.655  0.008   99.468  660.326
alcohol          -38.083  93.032 -0.409  0.682 -220.424  144.257
deadkids          36.833  51.394  0.717  0.474  -63.896  137.563
nprenatal        -14.009   6.785 -2.065  0.039  -27.307   -0.711
mrace            -111.01 102.289 -1.085  0.278 -311.493   89.474
frace             13.212  99.327  0.133  0.894 -181.466   207.89
fage              -0.023   2.859 -0.008  0.994   -5.625     5.58
fedu              -1.181 

**Interpretação:**

* mage (idade da mãe):
  * point_estimate: -11.047 (pvalue: 0.042)
  * Isso sugere que, para cada aumento de um ano na idade da mãe, o efeito negativo de fumar durante a gravidez no peso ao nascer diminui em 11.047 gramas (ou seja, o efeito se torna mais negativo).
  * Há evidência estatística de que a idade da mãe influencia o efeito de fumar durante a gravidez sobre o peso ao nascer.

* medu (educação materna):
  * point_estimate: 26.091 (pvalue: 0.015)
  * Para cada ano adicional de educação da mãe, o efeito negativo de fumar durante a gravidez no peso ao nascer é reduzido em 26.091 gramas (o efeito negativo é mitigado).
  * A educação materna parece reduzir o impacto negativo de fumar durante a gravidez.

* foreign (se a mãe é estrangeira):
  * point_estimate: 435.58 (pvalue: 0.001 (altamente significativo))
  * Interpretation: Mães estrangeiras têm um efeito tratamento condicional que é 435.58 gramas maior do que o de mães não estrangeiras.
  * A origem estrangeira da mãe está associada a uma redução significativa do efeito negativo de fumar durante a gravidez.

* Outras variáveis: Algumas covariáveis não são estatisticamente significativas (pvalue > 0.05), indicando que não há evidência suficiente para afirmar que essas covariáveis influenciam o efeito do tratamento.

* CATE Intercept Results (Resultados do Intercepto do CATE):
  * cate_intercept: 7.624
  * Este é o valor base do efeito tratamento condicional quando todas as covariáveis estão em zero. Como zero pode não ser um valor interpretável para algumas covariáveis (por exemplo, idade da mãe), o intercepto isoladamente pode não ter uma interpretação prática direta.

* OBS: Covariáveis contínuas vs. categóricas: Para variáveis contínuas (como mage), o coeficiente representa a variação no efeito do tratamento por unidade de aumento na covariável. Para variáveis binárias (como foreign), o coeficiente representa a diferença no efeito do tratamento entre os grupos (por exemplo, estrangeiras vs. não estrangeiras).


**Resumo:**

* O modelo estima que o efeito do tratamento (fumar durante a gravidez) sobre o peso ao nascer não é constante, mas varia linearmente com as covariáveis $X$. O sinal e magnitude dos coeficientes indicam a direção e a intensidade com que cada covariável afeta o efeito do tratamento.
* O impacto de fumar durante a gravidez no peso ao nascer não é o mesmo para todas as mães; varia de acordo com características como idade, educação e nacionalidade.
* Os resultados sugerem que políticas públicas visando reduzir o tabagismo durante a gravidez podem ser mais eficazes se levarem em consideração essas características. Por exemplo, focar em mães mais jovens ou com menor nível educacional.

**Como estimar o efeito do tratamento para um indivíduo com uma característica específica?**

Para calcular o efeito tratamento condicional ($\theta(X)$) para um conjunto específico de covariáveis, você pode usar a fórmula:

$$ \hat{\theta}(X) = X'\hat{\beta} + \hat{\theta}_{\text{intercept}}$$

Por exemplo, suponha que você queira calcular o efeito para uma mãe com as seguintes características:

In [65]:
X_new = pd.DataFrame({
    'casada': [1],
    'mage': [25],
    'medu': [12],
    'fhisp': [0],
    'mhisp': [0],
    'foreign': [0],
    'alcohol': [1],
    'deadkids': [0],
    'nprenatal': [10],
    'mrace': [1],
    'frace': [1],
    'fage': [30],
    'fedu': [10]
})

# Certificar-se de que as colunas correspondem às usadas no modelo
X_new = pd.get_dummies(X_new, drop_first=True)

# Calcular o efeito e obter a inferência
effect_inf = estimator.effect_inference(X_new)
print(effect_inf.summary_frame())


   point_estimate   stderr  zstat  pvalue  ci_lower  ci_upper
X                                                            
0         -253.02  101.255 -2.499   0.012  -451.476   -54.564


Para o perfil de mãe especificado, fumar durante a gravidez está associado a uma redução média de aproximadamente 253.02 gramas no peso ao nascer do bebê. Logo, há evidência estatística significativa de que fumar durante a gravidez está associado a uma redução substancial no peso ao nascer do bebê.

**Classes do DML**

**Modelos Lineares**

* LinearDML: usa um **modelo linear** final **não regularizado** e funciona essencialmente apenas quando o vetor de características $X$ é de baixa dimensão. Oferece intervalos de confiança por meio de argumentos de normalidade assintótica. Também é possível construir intervalos de confiança baseados em bootstrap definindo inference='bootstrap' (Chernozhukov, 2016).
* SparseLinearDML: é uma extensão do LinearDML que usa **regularização L1** para lidar com a alta dimensionalidade de $X$. Usa uma implementação do algoritmo DebiasedLasso, propriedades de normalidade assintótica do DebiasedLasso, esta classe também oferece intervalos de confiança assintoticamente normais (Chernozhukov, 2017; Chernozhukov, 2018). 
* KernelDML: é uma classe que usa **kernel ridge regression** para estimar o efeito do tratamento (RKHS - Nie, 2017). Ela aproxima qualquer função no RKHS criando recursos aleatórios de Fourier. Em seguida, executa um modelo final regularizado ElasticNet. Além disso, dado que usamos Recursos aleatórios de Fourier, esta classe assume um kernel RBF.


**Modelos Não-Lineares**

* NonParamDML: não faz nenhuma suposição sobre o modelo de efeito para cada resultado $i$. No entanto, ele se aplica somente quando o tratamento é binário ou unidimensional contínuo.
* CausalForestDML: usa uma Floresta Causal como um modelo final (Wager, 2018; Athey, 2019). Este estimador oferece intervalos de confiança via *Bootstrap-of-Little-Bags* conforme descrito em (Athey, 2019) . Usando esta funcionalidade, também podemos construir intervalos de confiança para o CATE.




**Modelos mais flexiveis (Não-Lineares)**

Se a heterogeneidade do efeito não tiver uma forma linear, então essa abordagem anterior não é válida. Pode-se então querer criar caracterizações mais complexas, em cujo caso o problema pode se tornar com dimensões muito altas para para OLS. 
* O *SparseLinearDML* pode lidar com essas configurações por meio do uso do Lasso desviado. 
* O *CausalForestDML* não precisa de caracterização explícita e aprende os modelos CATE não lineares baseados em floresta, automaticamente. 

Como há não lineariedade, não conseguimos interpretar os coeficientes diretamente. Conseguimos o ATE, e apenas podemos interpretar o efeito do tratamento para um indivíduo com uma característica específica.

In [82]:
from econml.dml import NonParamDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor

# Create the NonParamDML estimator
estimator_NonParam = NonParamDML(
    model_y=RandomForestRegressor(n_estimators=100, random_state=123),
    model_t=RandomForestClassifier(n_estimators=100, random_state=123),
    model_final=GradientBoostingRegressor(n_estimators=100, random_state=123),
    discrete_treatment=True,
    random_state=123
)

# Ajustar o modelo com inferência por bootstrap
estimator_NonParam.fit(y, D, X=X, inference='bootstrap')

# Obter a inferência do ATE
ate_inf = estimator_NonParam.ate_inference(X=X)

# Exibir o resumo da inferência
print(ate_inf.summary())

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
  -197.671     788.447 -0.251  0.802     -1742.998      1347.655
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
   430.33        -798.482         367.699
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
     898.238      -1312.975        967.929
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


In [92]:
# Obter as estimativas pontuais do CATE em X
cates = estimator_NonParam.effect(X)
cate_inf = estimator_NonParam.effect_inference(X)
# Obter o resumo das estimativas do CATE
print(cate_inf.summary_frame().head())

   point_estimate   stderr  zstat  pvalue  ci_lower  ci_upper
X                                                            
0        -284.299  150.622 -1.887    0.00  -661.645   -74.625
1         177.621  276.689  0.642    0.11  -360.993   665.202
2           9.170  252.484  0.036    0.39  -419.417   461.940
3        -227.112  100.160 -2.267    0.02  -398.421   -29.809
4        -428.132  291.145 -1.471    0.08  -991.964   156.033


No caso do NonParamDML, não é possível obter uma tabela semelhante com coeficientes para cada covariável. Isso se deve à natureza não paramétrica do estimador, que não assume uma forma funcional específica para a relação entre as covariáveis e o efeito do tratamento.

In [ ]:
# Definir as características específicas
X_new = pd.DataFrame({
    'casada': [1],
    'mage': [25],
    'medu': [12],
    'fhisp': [0],
    'mhisp': [0],
    'foreign': [0],
    'alcohol': [1],
    'deadkids': [0],
    'nprenatal': [10],
    'mrace': [1],
    'frace': [1],
    'fage': [30],
    'fedu': [10]
})

# Converter variáveis categóricas em dummies, se aplicável
X_new = pd.get_dummies(X_new, drop_first=True)

# Calcular a estimativa pontual do efeito
cate_new = estimator_NonParam.effect(X_new)

# Obter a inferência estatística
effect_inf = estimator_NonParam.effect_inference(X_new)

# Exibir o resumo da inferência
print(effect_inf.summary_frame())

   point_estimate   stderr  zstat  pvalue  ci_lower  ci_upper
X                                                            
0        -194.172  156.984 -1.237    0.11  -506.135     116.7


In [107]:
from econml.dml import CausalForestDML

# Criar o estimador ForestDML
estimator_Forest = CausalForestDML(
    model_y=RandomForestRegressor(n_estimators=100, random_state=123),
    model_t=RandomForestClassifier(n_estimators=100, random_state=123),
    discrete_treatment=True,
    random_state=123
)

# Ajustar o modelo
estimator_Forest.fit(y, D, X=X)

# Obter a inferência do ATE
ate_inf_forest = estimator_Forest.ate_inference(X=X)

# Exibir o resumo da inferência
print(ate_inf_forest.summary())

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
  -214.954     186.112 -1.155  0.248      -579.728       149.819
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
  143.065         -494.49          58.236
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
     234.746       -702.526        231.413
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


In [108]:
# Definir as características específicas
X_new = pd.DataFrame({
    'casada': [1],
    'mage': [25],
    'medu': [12],
    'fhisp': [0],
    'mhisp': [0],
    'foreign': [0],
    'alcohol': [1],
    'deadkids': [0],
    'nprenatal': [10],
    'mrace': [1],
    'frace': [1],
    'fage': [30],
    'fedu': [10]
})

# Converter variáveis categóricas em dummies, se aplicável
X_new = pd.get_dummies(X_new, drop_first=True)

# Calcular a estimativa pontual do efeito
cate_new = estimator_Forest.effect(X_new)

# Obter a inferência estatística
effect_inf = estimator_Forest.effect_inference(X_new)

# Exibir o resumo da inferência
print(effect_inf.summary_frame())

   point_estimate  stderr  zstat  pvalue  ci_lower  ci_upper
X                                                           
0        -192.011  94.156 -2.039   0.041  -376.554    -7.468


**Como podemos avaliar o desempenho do modelo CATE?**

Cada uma das classes DML tem um atributo score_ depois de serem ajustadas. Então, é possível acessar esse atributo e comparar o desempenho em diferentes parâmetros de modelagem (quanto menor a pontuação, melhor):

## Considerações Finais

Vejamos um resumo do que foi visto.

* O principal objetivo do DML é ajustar e remover a variável de confusão de forma que a variável de interesse (tratamento) e o resultado fiquem "ortogonais" ou "independentes".
* DML combina métodos de aprendizado de máquina com técnicas econométricas para estimar efeitos causais.
* A técnica geralmente envolve a aplicação de aprendizado de máquina para prever tanto o tratamento quanto o resultado usando as variáveis observáveis de confusão, e então os resíduos dessas previsões são utilizados em um segundo estágio para estimar o efeito causal.
  * Primeiro Estágio: Aplicar modelos de aprendizado de máquina para prever a variável de tratamento e o resultado.
  * Segundo Estágio: Utilizar os resíduos dessas previsões em um modelo de regressão para estimar o efeito causal.
